# M1 Metric Analysis

The tabular results of evaluating the M1 footprint metric for the RGPS survey design and all science cases can be found in the metric_results/ directory.  The goal of this notebook is to explore those results in graphical form. 

In [1]:
from os import path, getcwd
from sys import path as pythonpath
pythonpath.append(path.join(getcwd(), '..'))
from astropy.table import Table, Column, vstack
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import config_utils
import regions
import plot_metric_results
import visualization_utils
import numpy as np
from astropy import units as u 
from mw_plot import MWSkyMap

## Setup

First we need to load some of the configuration information for the metric simulation for future reference.

In [2]:
sim_config = config_utils.read_config(path.join(getcwd(), '..', 'config', 'sim_config.json'))

In [3]:
# Load the survey regions
all_survey_regions = regions.load_regions_from_file(sim_config, path.join(sim_config['root_dir'], 'region_data', 'rgps_survey_regions.json'))

In [4]:
# Load definitions of science cases and extract the set of topical categories
science_cases = config_utils.read_config(path.join(getcwd(), '..', 'config', 'rgps_science_cases.json'))

science_categories = []
for author, info in science_cases.items():
    if info['ready_for_use'] and info['category'] not in science_categories:
        science_categories.append(info['category'])
science_categories

['wide_area',
 'time_domain',
 'SFR',
 'solar_system',
 'high_cadence',
 'globular_clusters',
 'molecular_clouds',
 'AGN',
 'open_clusters',
 'variable_stars',
 'galactic_center',
 'keyholes',
 'stellar_spectra',
 'extinction',
 'novae',
 'ISM',
 'YSO',
 'GW']

## Useful functions

In [5]:
def select_results(metric_results, author_list, optic_list, survey_list):
    """
    Function to down-select the table of metric results by the name of the author of a science case, optic and survey design.
    """

    rows = []
    for entry in metric_results:
        if entry['Science_case'] in author_list \
            and entry['Optic'] in optic_list \
                and entry['Survey_strategy'] in survey_list:
            rows.append(entry)
            
    sub_results = Table(rows=rows, names=metric_results.colnames)

    return sub_results

## M1 Survey Footprint metric results 

The M1 metric evaluates the overlap between the survey regions requested by the different science cases and those actually included in each survey design.  Note that if a science case or strategy includes multiple regions per filter, the metric the sums the overlapping HEALpixel area for all regions.  

This metric is relevant to all science cases proposed though for different reasons.  For example, the most obvious comparison to make is between the survey region and the wide-area surveys proposed.  But the time domain fields selected are also of interest.  

Since including all proposals in a single graphic would make a very complex plot, the M1 metric is evaluated for the different science categories.  

### Regions of interest included in the RGPS Wide-Area Survey Footprint

We load the M1 results for each category of science, and plot heatmaps of the metric results for all proposals within that category.

In [6]:
strategy_name = 'wide_area'
metric_name = 'M1_%pix'
metric_label = '% area of science region in survey'

for category in science_categories:

    m1_results_file = path.join(getcwd(), '..', 'metric_results', 'M1_survey_footprint_' + category + '_results.txt')
    m1_results = Table.read(m1_results_file, format='ascii')

    plot_file = path.join(sim_config['root_dir'], 'metric_results', 'm1_results_heatmap_' + category + '.png')
    plot_metric_results.plot_metric_optic_heatmap(sim_config, m1_results, strategy_name, category,
                                  metric_name, metric_label, plot_file) 
    